In [3]:
from pandas import read_csv
from matplotlib import pyplot as plt
import sys
import numpy as np
import pandas as pd
from datetime import datetime

exer="1"

file_path = "C:\\Users\\lopes\\Desktop\\Apple watch\\Exercicio"

users1=["AntonioC","GusC","DarC"]

repetition=["R1","R2","R3","R4"]

pesos=["3kg","5kg"] 

tW=list(np.arange(-2.5,2.5+0.125,0.05))

tStep=0.1

def getMedian(vs):
    if (len(vs)==0):
        return 0
    return (np.median(vs))

X = []
Y = []
Find_Offset = []

for ixa,ps in enumerate(pesos):
        for ixu,us in enumerate(users1):             
            for ixc,cl in enumerate(repetition):       
                print("the user user is ",us," try nº ",cl," peso: ",ps)
                data_input = pd.read_csv(file_path+exer+"\\"+us+exer+"_"+cl+"_"+ps+".txt", skiprows=1, header=None, sep='\s+',
                                         names=["TimeStep","Dias","Segundos","XACC","YACC","ZACC","XGYR","YGYR","ZGYR"])
                tMax_input= data_input["Segundos"].iloc[-1]
                data_output = pd.read_excel(file_path+exer+"\\"+us+exer+"_"+cl+"_"+ps+".xlsx",header=0,sheet_name='Serie 1')
                tMax_output = data_output["Tiempo (s)"].iloc[-1]
                offset_scene = float(open(file_path+exer+"\\"+us+exer+"_"+cl+"_"+ps+".txt").readline().split('=')[1])
                print("Offset Inicial = ",offset_scene)
                
                print("Tempo Input = ",tMax_input," Tempo Output = ",tMax_output)
                tMaxMin=min(tMax_input,tMax_output)
        
                Xoffset = []
                Yoffset = []
                Timeoffset = []
        
                for t in np.arange(0-min(tW),tMaxMin-max(tW),tStep): #len = 131
                    xi=[]
                    yi=[]
                    Find_Offset1 = []
                       
                    xi1=[]
                    xi2=[]
                    xi3=[]
                    xi4=[]
                    xi5=[]
                    xi6=[]
                    Tempos = []
                    Find_Offset2 = []
                    for it in range(0,len(tW)-2,1): #len = 50
        
                            tr=t
        
                            t0=tr+tW[it]
                            tN=tr+tW[it+1]                                         
                            ag1=data_input.iloc[np.where((data_input["Segundos"] >= t0+offset_scene ) & (data_input["Segundos"] <=  tN+offset_scene))]
        
                            Tempos.append(t0)
                            Tempos.append(tN)
        
                            
                            xi1.append(getMedian(ag1.iloc[:,3]))
                            xi2.append(getMedian(ag1.iloc[:,4]))
                            xi3.append(getMedian(ag1.iloc[:,5]))
                            xi4.append(getMedian(ag1.iloc[:,6]))
                            xi5.append(getMedian(ag1.iloc[:,7]))
                            xi6.append(getMedian(ag1.iloc[:,8]))
                            Find_Offset1.append(t0)
                            
                    ag2=data_output.iloc[np.where((data_output["Tiempo (s)"] >= Tempos[0]) & (data_output["Tiempo (s)"] <=  Tempos[1]))]
                    
                    xi.append(xi1)
                    xi.append(xi2)
                    xi.append(xi3)
                    xi.append(xi4)
                    xi.append(xi5)
                    xi.append(xi6)
                    yi.append(getMedian(ag2.iloc[:,5]))
  
                    Find_Offset2.append(Find_Offset1)
                 
                    # list of len 6,51 --> [[ACCX],[ACCY],[ACCZ],[GYRX],[GYRY],[GYRZ]]
                    X.append(np.transpose(xi))
                    Y.append(np.transpose(yi))
                    Find_Offset.append(np.transpose(Find_Offset2))
        
                    Xoffset.append(np.transpose(xi))
                    Yoffset.append(np.transpose(yi)) 
                    Timeoffset.append(np.transpose(Find_Offset2))
                    
                Xoffset = np.array(Xoffset)
                Yoffset = np.array(Yoffset) 
                Timeoffset = np.array(Timeoffset)
                print(Xoffset.shape)
        
                mincorr = []
                for a in range(0,Xoffset.shape[1]):
                        meta = prinUserSensorMatrix(Xoffset,Yoffset,Timeoffset,a,False,None)
                        mincorr.append(meta)
                print(min(mincorr))
                #prinUserSensorMatrix(Xoffset,Yoffset,Timeoffset,0,True,min(mincorr)[1])
                #plt.show()
                print("----------------------------------")
                                     #                            it = 49              """             it = 0
X=np.array(X)                        #matrix t=1   [[ACCX,ACCY,ACCZ,GYRX,GYRX,GYRX],          , [ACCX,ACCY,ACCZ,GYRX,GYRX,GYRX]]
Y=np.array(Y)                        #       """
Find_Offset=np.array(Find_Offset)    #       t=max [[ACCX,ACCY,ACCZ,GYRX,GYRX,GYRX],          , [ACCX,ACCY,ACCZ,GYRX,GYRX,GYRX]]


print(X.shape,Y.shape,Find_Offset.shape)

the user user is  AntonioC  try nº  R1  peso:  3kg
Offset Inicial =  1.4000000000000012
Tempo Input =  37.525  Tempo Output =  34.684
(296, 101, 6)
(-0.9720109190520632, 0.0, 29.50000000000003)
----------------------------------
the user user is  AntonioC  try nº  R2  peso:  3kg
Offset Inicial =  1.7000000000000015
Tempo Input =  37.574  Tempo Output =  34.312


KeyboardInterrupt: 

In [2]:
def prinUserSensorMatrix(X_matrix,Y_matrix,Time,Noffset,on,mark_x=None):
    df = pd.DataFrame()
    ts=[]
    ls=[]

    for t in range(0,X_matrix.shape[0]):  #tempo = len(np.arange(0-min(tW),tMaxMin-max(tW),tStep))
        ts = Time[:,:,0]
        ts=ts[:,Noffset]
    df['Time']=ts
    
    for s in range(0,X_matrix.shape[2]):
        ls.append("sensor-"+str(s))
        vs=X_matrix[:,:,s]                   #vs.shape = (85,49) linhas são os dados de cada sensor no tempo t, para os it diferentes
        vs=vs[:,Noffset]                    # para cada sensor e tempo t, o primeiro tempo it 
        df["sensor-"+str(s)]=vs

    ls.append("Velocity m/s")
    ys=Y_matrix[:,0]
    df["Velocity m/s"]=ys

    if on:
        axes = df.plot(x="Time", y=ls, subplots=True, legend=True, figsize=(15,15))
        if mark_x is not None:
            for ax in axes:
                ax.axvline(x=mark_x, color='r', linestyle='--')
    
    correlation = df[ls[5]].corr(df[ls[6]])
    
    #plt.tight_layout() 
    #plt.savefig('sensor_data_plots_Dentro.png')
    
    return correlation,ts[0],ts[-1]
    
#prinUserSensorMatrix(X,Y,Find_Offset,30,True,None)